In [1]:
#The Big Heist - Project

In [2]:
#imports
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [3]:
df_banks = pd.read_csv('data/bank_data.csv')

df_banks

,id,x_coordinate,y_coordinate,money,time (hr)
0,0,2.851925,1.201712,29700,0.273403
1,1,4.150372,-2.654334,6500,0.064040
2,2,-1.494092,-1.230419,89400,0.127458
3,3,1.271326,-0.088520,96100,1.315029
4,4,2.471113,-0.592810,41100,0.164393
...,...,...,...,...,...
9995,9995,-2.472127,4.787304,2100,0.494904
9996,9996,1.034968,-4.321240,54900,0.492866
9997,9997,-0.512720,-2.949173,32200,0.933033
9998,9998,2.626841,-3.482923,5100,0.551312


In [4]:
speed = 30.0 # km/hr
maxTime = 23.96
count=0
def distance(x, y):
    p1 = np.sqrt(sum((np.subtract(x, y))**2))
    return p1

def wrapper(): #holds the arrays
    money_bag = []
    bank_id_array = []
    netTime_array = []
    maxTime = 24.0

    def robbing(): #robs the banks
        df = df_banks.copy()
        N = len(df['id'])
        loc = np.random.randint(N)
        home = np.array([0.0,0.0])
        origin = np.array([df['x_coordinate'][loc], df['y_coordinate'][loc]])
        home_time = distance(origin, home)/speed
        robTime = df['time (hr)'][loc]
        netTime = home_time + robTime
        newLoc = loc
        bank_id = []
        net_money = 0
        while netTime < maxTime:
            netTime -= home_time
            # df.drop([newLoc], inplace=True)
            distance_array = [distance(origin, np.array([df['x_coordinate'][i], df['y_coordinate'][i]])) for i in df['id']]
            net_time = np.array(distance_array)/speed + df['time (hr)'] 
            time_money_ratio = [net_time[i]/df['money'][i] for i in df['id']]
            timeWithout = np.array(time_money_ratio)[time_money_ratio != np.amin(time_money_ratio)]        
            newLoc = np.where(np.min(timeWithout) == timeWithout)[0][0]
            net_money += df['money'][newLoc]
            home_time = distance(home, (df['x_coordinate'][newLoc], df['y_coordinate'][newLoc]))/speed
            netTime += net_time[newLoc] + home_time
            df.at[newLoc, 'money'] = 1e-2
            origin = np.array([df['x_coordinate'][newLoc], df['y_coordinate'][newLoc]])
            bank_id.append(newLoc)
        # print(bank_id, netTime, net_money)
        return bank_id, netTime, net_money

    iter = 0
    Nmax = 15#max number of itereations through the bank ids
    found = False
    while iter < Nmax:#run the function until we find an acceptable answer
        bank_id, netTime, net_money = robbing()
        bank_id_array.append(bank_id)
        netTime_array.append(netTime)
        money_bag.append(net_money)
        if abs(netTime - maxTime) > 0.02:
            iter+=1
        else:
            found = True
            break
    if found == True:
        return bank_id_array[~0], netTime_array[~0], money_bag[~0]
    else:
        loc = np.where(netTime_array == np.min(netTime_array))[0][0]

        return bank_id_array[loc], netTime_array[loc], money_bag[loc]

In [9]:
%%time
warnings.filterwarnings("ignore")
bank_id_array, netTime_array, money_bag = wrapper()
print(f"The banks looted: {bank_id_array} \n Time to loot: {netTime_array}  \n Money Earned: {money_bag}")

The banks looted: [4498, 4788, 5155, 8469, 9240, 4696, 7063, 5933, 487, 8524, 433, 5135, 8562, 557, 6253, 4762, 299, 3193, 6468, 8524, 7544, 9545, 8230, 9227, 9401, 3297, 8206, 2656, 9048, 5610, 9195, 4987, 3798, 6097, 7064, 4789, 8435, 5563, 865, 444, 7559, 6740, 7074, 7258, 3026, 8287, 517, 7700, 4983, 7559, 8848, 4498, 7688, 5294, 8525, 4499, 8689, 9652, 9545] 
 Time to loot: 24.10149822720609  
 Money Earned: 2610000
Wall time: 2min 34s
